In [1]:
import pandas as pd
import json
import requests
from python_secrets import *

In [115]:
df = pd.read_csv('incident2.csv')
df.head()

,block_number,user_tx,fees
0,19412019,0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8...,1.861322e+16
1,19412030,0x354e8386267ca643793de913739df3f9680895776f3a...,4.694753e+16
2,19412041,0xee8fd2c76181afa14ca0da158e0a01bba2d3df8e62c5...,6.182506e+16
3,19412043,0x5f3954cc3cb4fbb88803a910d852aab6566af2866acf...,1.283444e+17
4,19412043,0xbd9c86df3327871d630a0921c65cb7bec9b0c199d80c...,3.640861e+16


In [116]:
tx_hash_list = [x for x in df[' user_tx'].to_list() if pd.notnull(x)]

# Infuria

In [119]:
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionReceipt",
  "params": [
    "0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581"
  ],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

columns_no_duplicates = [e for e in response.json()['result'].keys() if e not in ('logs', 'logsBloom')]
df_infuria = pd.DataFrame.from_dict(response.json()['result'])[columns_no_duplicates]
df_infuria = df_infuria.drop_duplicates()
df_infuria

for tx_hash in tx_hash_list[1:]:
    payload = json.dumps({
      "jsonrpc": "2.0",
      "method": "eth_getTransactionReceipt",
      "params": [
        tx_hash
      ],
      "id": 1
    })
    headers = {
      'Content-Type': 'application/json'
} 

    response = requests.request("POST", url, headers=headers, data=payload)

    if response.ok:
      df_temp = pd.DataFrame.from_dict(response.json()['result'])[columns_no_duplicates]
      df_temp = df_temp.drop_duplicates()
      df_infuria = pd.concat([df_infuria, df_temp])
    
    else: 
       print(f"error code {response.status_code} for transaction {tx_hash}")


# W3